#Chapter 22

Language Modeling toy example

i like this black cat.

prodict next word

i like this [] <= black


vocab: "100 words"

vocab
word -> index
i -> 2
like -> 45
this -> 30
black -> 55
cat -> 10
. -> 1

our goal is to prodict 
"i like this black cat []" <= .

input => i like this black cat
target => like this black cat.

x(t){Wx}  ->[tanh]->[softmax]->y^(t)
h(t-1){Wy}->[tanh]->h(t)


h0 = numFeatures*1 = 100
[tanh] = no. units = 50
Wx = no. units * num Features = 50*100
Wy = no. units * vocab size = 50*80
Wh = no. units * history len = 50*50
h(t) = 50 * 1

we need to convert the input history into tanh form this can be done via 

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import sys


# function to get one hot encoding 
def getOneHot(idx):
    one_hot = np.zeros((vocabSize,1))
    one_hot[idx] = 1
    return one_hot

def stepForward(xt,Wx,Wh,outputWeight_Wy,prevMemory):
    x_frd = torch.matmul(Wx,torch.from_numpy(xt))
    h_frd = torch.matmul(Wh,prevMemory)
    ht = torch.tanh(x_frd+h_frd)
    yt_hat = F.softmax(torch.matmul(outputWeight_Wy,ht),dim=0)
    return ht,yt_hat

def fullForwardRNN(X,Wx,Wh,outputWeight_Wy,prevMemory):
    y_hat = []
    for t in range(len(X)):
        ht,yt_hat = stepForward(X[t],Wx,Wh,outputWeight_Wy,prevMemory)
        prevMemory = ht
        y_hat.append(yt_hat)
    return y_hat  

def computeLoss(y,y_hat):#y => 1 hope vectors , y_hat => prodictions of model
    loss = 0
    for yi,yi_hat in zip(y,y_hat):
        Li = -torch.log2(yi_hat[yi==1])
        loss += Li
    return loss/len(y)

def updateParams(Wx,Wh,Wy,dWx,dWh,dWy,lr):
    with torch.no_grad():
        Wx -= lr*dWx
        Wh -= lr*dWh
        Wy -= lr*dWy
    return Wx,Wh,Wy

def trainRNN(X,y,Wx,Wh,Wy,prevMemory,lr,nepoch):
    losses = []
    for epoch in range(nepoch):
        y_hat = fullForwardRNN(X,Wx,Wh,Wy,prevMemory)
        loss = computeLoss(y,y_hat)
        loss.backward()
        losses.append(loss)
        print("Loss after epoch=%d: %f" %(epoch,loss))
        sys.stdout.flush()
        dWx = Wx.grad.data
        dWh = Wh.grad.data
        dWy = Wy.grad.data
        Wx,Wh,Wy = updateParams(Wx,Wh,Wy,dWx,dWh,dWy,lr)
        Wx.grad.data.zero_()
        Wh.grad.data.zero_()
        Wy.grad.data.zero_()
    return Wx,Wh,Wy,losses

#-------------------------------------------------
# Magic Values and dont know where they came from
# cannot find where he picked this number from
#-------------------------------------------------
# number of features
numFeatures = 100
# number of units in a Neural Network cell
numUnits = 50



# this is faked but we would load in a document of some kind
# build a dictionary of every word and assign an index
# given a sentence change the sentence from text to an array of indexes

vocabSize = 80
inputString = [2,45,30,55,10]
outputString = [45,30,55,10,1]

# each index in our vocab needs to have some embedding
# an array of values that has some meaning
# the meaning and how they are calculated was not covered
# but this should be calculated when getting the vocab
word_embeddings = []
for i in range(len(inputString)):
    x = np.random.randn(numFeatures,1)
    word_embeddings.append(x)






In [2]:



h0 = torch.tensor(np.zeros((numUnits,1)))
historyWeight_Wh = torch.tensor(np.random.uniform(0,1,(numUnits,numUnits)),requires_grad=True)
featureWeight_Wx = torch.tensor(np.random.uniform(0,1,(numUnits,numFeatures)),requires_grad=True)
outputWeight_Wy = torch.tensor(np.random.uniform(0,1,(vocabSize,numUnits)),requires_grad=True)





ht,yt_hat = stepForward(word_embeddings[0],featureWeight_Wx,historyWeight_Wh,outputWeight_Wy,h0)






torch.Size([50, 50]) torch.Size([50, 100]) torch.Size([80, 50]) torch.Size([50, 1])


tensor(1.0000, dtype=torch.float64, grad_fn=<SumBackward0>)

In [3]:


y_hat = fullForwardRNN(word_embeddings,featureWeight_Wx,historyWeight_Wh,outputWeight_Wy,h0)



torch.Size([80, 1])

In [ ]:


y = []
for idx in outputString:
    y.append(getOneHot(idx))

#print(computeLoss(y,y_hat))



Wx,Wh,Wy,losses = trainRNN(word_embeddings,y,featureWeight_Wx,historyWeight_Wh,outputWeight_Wy,h0,0.001,100)